###  Wrangling, analyzing and visualizing all h\*ckin' good boys and girls !!  

Twitter user [dog_rates](https://twitter.com/dog_rates) AKA [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs)

###### For more on Doggolingo check link below 
[Dogs Are Doggos: An Internet Language Built Around Love For The Puppers](https://www.npr.org/sections/alltechconsidered/2017/04/23/524514526/dogs-are-doggos-an-internet-language-built-around-love-for-the-puppers)

<a class="anchor" id="jmp2lnks"></a>

<img src="https://drive.google.com/uc?id=1q27qppQ80W_giOg6vN0ngcvl_30Qgpcp" width="100">

* <a href="#gathereddataframes" style='color:#8D38C9'>Gathered DataFrames</a>
* <a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>
* <a href="#cln" style='color:#8D38C9'>Data cleaning process</a>
* <a href="#masterdataframes" style='color:#8D38C9'>Master DataFrame(s)</a>
* <a href="#analysis" style='color:#8D38C9'>Analysis and visualization</a>

## Gather

We will be gathering data from different sources and in different formats, the aim is to have a master DataFrame(s) including several details about tweets and dogs to help us in latter analysis and visualization.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sys
import requests
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
from bs4 import BeautifulSoup as bs
from functools import reduce
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import squarify
import random
import warnings;warnings.filterwarnings("ignore")
%matplotlib inline


<a class="anchor" id="twtarch"></a>

In [2]:
# load dataframe
twt_arch_df=pd.read_csv('twitter-archive-enhanced.csv')

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="imgprd"></a>

In [3]:
# download and save image-predictions file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)
with open("image-predictions.tsv", mode='wb') as file:
    file.write(response.content)

In [4]:
# load dataframe
img_predicts_df=pd.read_csv('image-predictions.tsv', sep='\t')

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="twtapi"></a>

In [5]:
# Uncomment to reproduce, use your own keys for accessing the API

# # Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# consumer_key = ''
# consumer_secret = ''
# access_token = ''
# access_secret = ''

# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True)

# tweet_ids = twt_arch_df.tweet_id.values

# # Query Twitter's API for JSON data for each tweet ID in the Twitter archive
# count = 0
# fails_dict = {}
# start = timer()
# # Save each tweet's returned JSON as a new line in a .txt file
# with open('tweet_json.txt', 'w') as outfile:
#     for tweet_id in tweet_ids:
#         count += 1
#         print(str(count) + ": " + str(tweet_id))
#         try:
#             tweet = api.get_status(tweet_id, tweet_mode='extended')
#             print("Success")
#             json.dump(tweet._json, outfile)
#             outfile.write('\n')
#         except tweepy.TweepError as e:
#             print(e)
#             fails_dict[tweet_id] = e
#             pass
# end = timer()
# print(end - start)

# save missing tweets, incase needed latter
# fails_df=pd.DataFrame(fails_dict.items())
# fails_df.to_csv('Missing Tweets', index=False)

In [6]:
# Read Tweet Json file and extract data

# list to append data for latter conversion into a datframe
df_list = []

with open ('tweet_json.txt', 'r') as file:
    for line in file:
        tweet_data=json.loads(line)
        t_id=tweet_data['id']
        rt_count=tweet_data["retweet_count"]
        fvt_count=tweet_data["favorite_count"]
        t_txt=tweet_data['full_text']
        df_list.append({
            'tweet_id':t_id,
            'retweet_count':rt_count,
            'favorite_count':fvt_count,
            'full_text':t_txt,
        })

# convert list into dataframe
twt_api_df = pd.DataFrame(df_list, columns=('tweet_id', 'retweet_count', 'favorite_count','full_text'))

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="dgbrd"></a>

In [ ]:
url = 'https://dogtime.com/dog-breeds/profiles'
# connect to url
response = requests.get(url)

# Work with HTML file directly without downloading:
soup = bs(response.content, 'lxml')

In [ ]:
# obtain all dog breeds details (name,image,info,links)

# list to append data for latter conversion into a datframe
dog_breed_list = []

# list of urls from which breed details will be extracted, this is obtained from reading main url HTML file and locating position of needed data
breed_url_list= ['article-crumbs group-letter letter-a js-letter-section paws',
                 'article-crumbs group-letter letter-b js-letter-section paws',
                 'article-crumbs group-letter letter-c js-letter-section paws',
                 'article-crumbs group-letter letter-d js-letter-section paws',
                 'article-crumbs group-letter letter-e js-letter-section paws',
                 'article-crumbs group-letter letter-f js-letter-section paws',
                 'article-crumbs group-letter letter-g js-letter-section paws',
                 'article-crumbs group-letter letter-h js-letter-section paws',
                 'article-crumbs group-letter letter-i js-letter-section paws',
                 'article-crumbs group-letter letter-j js-letter-section paws',
                 'article-crumbs group-letter letter-k js-letter-section paws',
                 'article-crumbs group-letter letter-l js-letter-section paws',
                 'article-crumbs group-letter letter-m js-letter-section paws',
                 'article-crumbs group-letter letter-n js-letter-section paws',
                 'article-crumbs group-letter letter-o js-letter-section paws',
                 'article-crumbs group-letter letter-p js-letter-section paws',
                 'article-crumbs group-letter letter-r js-letter-section paws',
                 'article-crumbs group-letter letter-s js-letter-section paws',
                 'article-crumbs group-letter letter-t js-letter-section paws',
                 'article-crumbs group-letter letter-v js-letter-section paws',
                 'article-crumbs group-letter letter-w js-letter-section paws',
                 'article-crumbs group-letter letter-x js-letter-section paws',
                 'article-crumbs group-letter letter-y js-letter-section paws',
                ]


# iterate through the list and extract required data
for clss in breed_url_list:
    dog_breed= soup.find('div', class_=clss).find_all(class_="list-item-title")
    dog_breed_info = soup.find('div', class_=clss).find_all(class_="list-item-img")
    dog_breed_img = soup.find('div', class_=clss).find_all(class_="list-item-breed-img")
    for text in dog_breed:
        dog_breed_list.append({'breed':text.text})
    for inf in dog_breed_info:
        dog_breed_list.append({'breed_info_link':inf.attrs['href']})
    for img in dog_breed_img:
        dog_breed_list.append({'breed_img_link':img.attrs['src']})  

# creat df and save data to csv file
breed_df = pd.DataFrame(dog_breed_list, columns=['breed', 'breed_info_link', 'breed_img_link'])
breed_df = breed_df.apply(lambda x: pd.Series(x.dropna().values))
breed_df.to_csv('dog_breed_info.csv', index=False)

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="dgbrdinfo"></a>

In [ ]:
# # This cell took sometime to run, 385 url to pharse. Uncomment to reproduce.
# # U can skip this step, it is just to extracts info text for display in the dataframe instead of going to the web site and reading from there
# # breed info links already extracted

# # info text links from which data will be extracted 

# url_array=breed_df.breed_info_link.values

# breed_info_text_list=[]
# count = 0

# # iterate 
# for i in url_array:
#     count +=1
#     print(str(count) + ": " + str(i))
#     response=requests.get(i)
#     soup=bs(response.content, 'lxml')
#     breed_info_text = soup.find('div', class_='breeds-single-intro').find('p').text
#     breed_info_text_list.append({i.split('/')[-1]:breed_info_text})
#     print("Success")

# # save
# breed_info_df = pd.DataFrame([(key, value) for i in breed_info_text_list for key, value in i.items()], columns=['breed','breed_info_text'])
# breed_info_df.to_csv('dog_breed_info_text.csv', index=False)

<a class="anchor" id="gathereddataframes"></a>
### Gathered DataFrames

* <a href="#twtarch" style='color:#8D38C9'>Twitter Archive</a>
* <a href="#imgprd" style='color:#8D38C9'>Image Predictions</a>
* <a href="#twtapi" style='color:#8D38C9'>Twitter API</a>
* <a href="#dgbrd" style='color:#8D38C9'>Dog Breed</a>
* <a href="#dgbrdinfo" style='color:#8D38C9'>Dog Breed Info</a>

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

## Assess

In [ ]:
# Adjusting  display options, increase width and row numbers to remove truncation
pd.options.display.max_colwidth=150
pd.options.display.max_rows=100
pd.options.display.large_repr='truncate'

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**

<a class="anchor" id="arcdf"></a>

In [ ]:
twt_arch_df.head()

In [ ]:
twt_arch_df.info()

In [ ]:
twt_arch_df.describe().round()

In [ ]:
twt_arch_df.tweet_id.duplicated().sum()

In [ ]:
# assessing incorrect rating scale for denominator, official is 10
fltrd_rtng_dnm=twt_arch_df[(twt_arch_df.rating_denominator!=10) & twt_arch_df.in_reply_to_status_id.isnull() & twt_arch_df.retweeted_status_id.isnull()].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1)
fltrd_rtng_dnm.head()

In [ ]:
fltrd_rtng_dnm.tweet_id.count()

In [ ]:
# assessing incorrect rating extraction
fltrd_rtng_num=twt_arch_df[(twt_arch_df.rating_numerator!=10) & (twt_arch_df.rating_denominator==10) & twt_arch_df.in_reply_to_status_id.isnull() & twt_arch_df.retweeted_status_id.isnull()].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1)
fltrd_rtng_num.sample(5)

In [ ]:
fltrd_rtng_num.tweet_id.count()

In [ ]:
# examine top ratings
fltrd_rtng_num.sort_values('rating_numerator', ascending=False).head()

In [ ]:
# identify magnitude of tweets having float rating scale noted above
twt_arch_df[twt_arch_df.text.str.contains('((?:\d+\.)\d+)\/(\d+)')].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1)

In [ ]:
# check for proper name extraction
fltrd_name=twt_arch_df[twt_arch_df.name != 'None'].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1)
fltrd_name

In [ ]:
# inspect unusual names
twt_arch_df.name.value_counts().to_frame().sort_index(ascending=False)

In [ ]:
# assess wrong extraction
twt_arch_df[twt_arch_df.text.str.contains('name')].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1).head()

In [ ]:
# assess wrong extraction
twt_arch_df[twt_arch_df.name.str.islower()].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1).head()

In [ ]:
# assess wrong stage extraction, substitute doggo in both conditions with other stages to switch display
twt_arch_df[twt_arch_df.text.str.contains('doggo') & ~(twt_arch_df.doggo != 'None')].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1).head()

In [ ]:
# assess duplicate stages
twt_arch_df[(twt_arch_df.doggo != 'None') & (twt_arch_df.floofer !='None')].index

In [ ]:
# assess duplicate stages
twt_arch_df[(twt_arch_df.doggo != 'None') & (twt_arch_df.pupper !='None')].index

In [ ]:
# assess duplicate stages
twt_arch_df[(twt_arch_df.doggo != 'None') & (twt_arch_df.puppo !='None')].index

In [ ]:
# assess duplicate stages
twt_arch_df[(twt_arch_df.doggo != 'None') & (twt_arch_df.pupper !='None') & (twt_arch_df.in_reply_to_status_id.isnull()) & (twt_arch_df.retweeted_status_id.isnull())].drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], axis=1)

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**

<a class="anchor" id="imgdf"></a>

In [ ]:
img_predicts_df.head()

In [ ]:
img_predicts_df.info()

In [ ]:
img_predicts_df.describe().round(5)

In [ ]:
img_predicts_df.jpg_url.duplicated().sum()

In [ ]:
# inspect duplicates
img_predicts_df[img_predicts_df.jpg_url.duplicated()].sort_values('tweet_id').head()

In [ ]:
# assess predictions credibility
img_predicts_df[img_predicts_df.p1_conf>=0.9]

In [ ]:
# assessing for false negative
img_predicts_df[~(img_predicts_df.p1_dog) & ~(img_predicts_df.p2_dog) & ~(img_predicts_df.p3_dog)]

In [ ]:
# assessing false positive
img_predicts_df[(img_predicts_df.p1_dog) & (img_predicts_df.p2_dog) & (img_predicts_df.p3_dog)]

In [ ]:
# assess predictions credibility
img_predicts_df[img_predicts_df.p1_conf==img_predicts_df.p1_conf.min()].append(img_predicts_df[img_predicts_df.p2_conf==img_predicts_df.p2_conf.min()]).append(img_predicts_df[img_predicts_df.p3_conf==img_predicts_df.p3_conf.min()]).round(5)

In [ ]:
# assess predictions credibility
img_predicts_df[img_predicts_df.p1_conf==img_predicts_df.p1_conf.max()].append(img_predicts_df[img_predicts_df.p2_conf==img_predicts_df.p2_conf.max()]).append(img_predicts_df[img_predicts_df.p3_conf==img_predicts_df.p3_conf.max()]).round(5)

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**

<a class="anchor" id="apidf"></a>

In [ ]:
twt_api_df.head()

In [ ]:
twt_api_df.info()

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**

<a class="anchor" id="brdf"></a>

In [ ]:
breed_df.head()

In [ ]:
breed_df.info()

In [ ]:
breed_df.breed.duplicated().sum()

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**

<a class="anchor" id="brinfdf"></a>

In [ ]:
# Load df from saved csv to avoid running scrapping code
breed_info_df=pd.read_csv('dog_breed_info_text.csv')
breed_info_df.head()

In [ ]:
breed_info_df.info()

In [ ]:
breed_info_df.breed.duplicated().sum()

<a class="anchor" id="assessmentoutcome"></a>
### Assessment outcome
<a class="anchor" id="qlt"></a>
* <a href="#qualityissues" style='color:#8D38C9'>A-Quality Issues</a>
    * <a href="#arcdf" style='color:#C45AEC'>A.1-Twitter Archive Dataframe:</a>
      * [A.1.1-Variables does not conform to our defined schema](#A1.1)
      * [A.1.2-Time stamp dtype not properly identified](#A1.2)
      * [A.1.3-Some tweets have misleading rating](#A1.3)
      * [A.1.4-Dog name column has several wrong input ('a','the','this','an'), missing names are misrepresented and sometimes not properly extracted from tweet text](#A1.4)
      * [A.1.5-Multiple dog stage occurrences for a single tweet](#A1.5)
      * [A.1.6-Dog stage not properly extracted and missing values are misrepresented](#A1.6)
    * <a href="#imgdf" style='color:#C45AEC'>A.2-Image predictions Dataframe:</a>
      * [A.2.1-Duplicated records, mainly related to retweets and replies](#A1.1)
      * [A.2.2-Image prediction results are somewhat misleading](#A2.2)
      * [A.2.3-Some breed names are incorrect and sometimes misleading](#A2.3)
    * <a href="#brdf" style='color:#C45AEC'>A.3-Breed Dataframe:</a>
      * [A.3.1-Breed column's naming convention is not unified](#A2.3)
    * <a href="#brinfdf" style='color:#C45AEC'>A.4-Breed Info Dataframe:</a>
      * [A.4.1-Breed column's naming convention is not unified](#A2.3)
    ----
<a class="anchor" id="tid"></a>
* <a href="#tidinessissues" style='color:#8D38C9'>B-Tidiness Issues</a>
    * <a href="#arcdf" style='color:#C45AEC'>B.1-Twitter Archive Dataframe:</a>
      * [B.1.1-Tweet text column include both tweet text and links](#B1.1)
      * [B.1.2-One variable in four columns. I.e: 'doggo', 'floofer' are different representations of a single variable 'Dog Stage'](#A1.6)
      * [B.1.3-Total entries does not match that in Image predictions Dataframe](#B1.3)
    * <a href="#imgdf" style='color:#C45AEC'>B.2-Image predictions Dataframe:</a>
      * [B.2.1-All breed information should be included in the Dataframe](#A2.3)


**<a href="#masterdataframes" style='color:#8D38C9'>Master Dataframe(s)</a>**

**<a href="#analysis" style='color:#8D38C9'>Analysis and visualization</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**


<a class="anchor" id="cln"></a>
## Clean

In [ ]:
# Making copies of gathered DataFrames
df_twt_arch_c=twt_arch_df.copy()
df_img_pred_c=img_predicts_df.copy()
df_twt_api_c=twt_api_df.copy()
df_breed_c=breed_df.copy()
df_breed_inf_c=breed_info_df.copy()

<a class="anchor" id="qualityissues"></a>
### Quality Issues

<a class="anchor" id="A1.1"></a>
#### A.1.1 - Variables does not conform to our defined schema
---
##### Define

A total of 181(retweets) and 78(replies) are included in both archive and prediction Dataframe(s). We'd like to analyze original ratings of dogs only, so having retweets and replies in the dataframe does not follow our defined schema. We'll:

- Identify non-null rows in all (in_reply_ / retweeted_) columns. this will also clear issue --> B.2.1 Duplicated records, mainly related to retweets and replies <--
- Remove these columns all together since they wont be used in our analysis.
- Bonus: Remove 'source' and 'expanded_urls' columns also since they won't be used in our analysis.

##### Code

In [ ]:
# filter and drop non-null rows, non null indicates that this tweet is either a reply or retweet
df_twt_arch_c=df_twt_arch_c[df_twt_arch_c.in_reply_to_status_id.isnull() & df_twt_arch_c.in_reply_to_user_id.isnull() & df_twt_arch_c.retweeted_status_id.isnull() & df_twt_arch_c.retweeted_status_user_id.isnull() & df_twt_arch_c.retweeted_status_timestamp.isnull()]
df_twt_arch_c.drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','source','expanded_urls'], axis=1, inplace=True)
df_twt_arch_c.reset_index(drop=True, inplace=True)

In [ ]:
# removing retweets and replies data from predictions dataframe
df_img_pred_c=df_img_pred_c[df_img_pred_c.tweet_id.isin(df_twt_arch_c.tweet_id)]
df_img_pred_c.reset_index(drop=True, inplace=True)

##### Test

In [ ]:
df_twt_arch_c.info()

In [ ]:
df_img_pred_c.info()

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="A1.2"></a>
#### A.1.2 - Time stamp dtype not properly identified
---
##### Define
- Change timestamp dtype to datetime
- Bonus: change header format of 'timestamp' and 'name' columns to be more representative

##### Code

In [ ]:
# convert dtype to date time and rename the header
df_twt_arch_c['timestamp']=pd.to_datetime(df_twt_arch_c['timestamp']).dt.tz_localize(None)
df_twt_arch_c.rename(columns={'timestamp': 'tweet_time_stamp', 'name': 'dog_name'}, inplace=True)

##### Test

In [ ]:
df_twt_arch_c.info()

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="A1.3"></a>
#### A.1.3 - Some tweets have misleading rating
---
##### Define

here we have 3 issues as follows:
- 17 tweets denominator is not matching the official rating scale of 10
- 5 out of 17 have rating not properly extracted due to the existence of multiple ratings in the tweet text
- 5 tweets have rating not properly extracted because the rating scale mentioned in tweet text is float not int

We'll clean the above issues in a reverse order:
- Apply correct rating if not properly extracted from tweet text
- Change denominator of 17 to 10 to match official rating scale of 10

##### Code

In [ ]:
# filter all tweets with wrong rating
wrong_rating_df1 = df_twt_arch_c[(df_twt_arch_c.rating_denominator!=10) & df_twt_arch_c.text.str.contains('((?:\d+\.)?\d+)\/(\d+)')]

# extract correct rating to a separate dataframe
correct_ratings_df = wrong_rating_df1.text.str.extractall('((?:\d+\.)?\d+)\/(\d+)').unstack()

# remove multiIndex and rename columns
correct_ratings_df.columns = correct_ratings_df.columns.droplevel()
correct_ratings_df.drop([0, 0], axis=1, inplace=True)
correct_ratings_df.dropna(inplace=True)
correct_ratings_df.columns=['rtng_num','rtng_dnm']

# apply correct rating
df_twt_arch_c.loc[df_twt_arch_c.index.isin(correct_ratings_df.index), 'rating_numerator'] = correct_ratings_df['rtng_num']

In [ ]:
# filter all tweets with float rating
wrong_rating_df2 = df_twt_arch_c[df_twt_arch_c.text.str.contains('((?:\d+\.)\d+)\/(\d+)')]

# extract correct rating to a separate dataframe
float_ratings_df = wrong_rating_df2.text.str.extract('((?:\d+\.)?\d+)\/(\d+)',expand=True)

# rename columns
float_ratings_df.columns=['rtng_num','rtng_dnm']

# apply correct rating and dtype
df_twt_arch_c.loc[df_twt_arch_c.index.isin(float_ratings_df.index), 'rating_numerator'] = float_ratings_df['rtng_num']
df_twt_arch_c.rating_numerator=pd.to_numeric(df_twt_arch_c.rating_numerator)

In [ ]:
df_twt_arch_c.loc[(df_twt_arch_c.rating_denominator!=10) ,'rating_denominator']=10

##### Test

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.index.isin(correct_ratings_df.index)]

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.index.isin(float_ratings_df.index)]

In [ ]:
df_twt_arch_c[df_twt_arch_c.rating_denominator!=10]

In [ ]:
df_twt_arch_c.info()

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="A1.4"></a>
#### A.1.4 - Dog name column has several wrong input ('a','the','this','an'), missing names are misrepresented and sometimes not properly extracted from tweet text
---
##### Define
Here we have 2 issues that we will address as follows:
- Change lower case words and 'None' inputs to NaN. Lower case words are always not dog names.
- Search and extract dog names that are already mentioned in tweet text.

##### Code

In [ ]:
# change all lower case words and and missing names to 'not mentioned'
df_twt_arch_c.loc[df_twt_arch_c.dog_name.str.islower(),'dog_name']= np.nan
df_twt_arch_c.loc[df_twt_arch_c.dog_name=='None','dog_name']= np.nan

# Search and extract all text following the word 'named'
index_1=df_twt_arch_c.text.str.extract(r'(named?\s\w+.\w+)').dropna()

# slice out names,dog names mentioned in two forms either following 'is' or 'named'
index_2=index_1[0].str.extract(r'(is?\s\w+)').dropna()
index_3=index_1[0].str.extract(r'^(named\s\w+)').dropna()

# apply correct names
df_twt_arch_c.loc[df_twt_arch_c.index.isin(index_2.index), 'dog_name']=index_2[0].str.slice(3)
df_twt_arch_c.loc[df_twt_arch_c.index.isin(index_3.index), 'dog_name']=index_3[0].str.slice(6)

# this tweet had name mentioned after 'is', so will change individually
df_twt_arch_c.loc[df_twt_arch_c.tweet_id==666739327293083650, 'dog_name']='Lugan'

##### Test

In [ ]:
df_twt_arch_c.dog_name.value_counts().sample(20)

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.index.isin(index_2.index)].head()

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.index.isin(index_3.index)].head()

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.tweet_id==666739327293083650]

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="A1.5"></a>
#### A.1.5 - Multiple dog stage occurrences for a single tweet
---
##### Define

- Delete wrong entries based on tweet text.
- Merge the multiple values into one for tweets having multiple dogs.
- Bonus: Fix missing dog names if already included in tweet text but referenced as 'not mentioned'

##### Code

In [ ]:
# filter and apply correct dog stage
df_twt_arch_c[(df_twt_arch_c.doggo != 'None') & (df_twt_arch_c.floofer !='None')].index
df_twt_arch_c.loc[161,'doggo']='None'

In [ ]:
# filter and apply correct dog stage
df_twt_arch_c[(df_twt_arch_c.doggo != 'None') & (df_twt_arch_c.pupper !='None')]

In [ ]:
# Changes to be done based on visual inspection of above filter:
# 358'doggo=None' - 446'doggo=None' - 416'Burke/Dexter pupper/doggo'  - 689'Lila/Maggie pupper/doggo' - 748'doggo' - 848'pupper/doggo' - 897'pupper/doggo'
df_twt_arch_c.loc[[358,446,536,562],'doggo']='None'
df_twt_arch_c.loc[[416,689],['dog_name','doggo','pupper']]=[('Burke/Dexter','pupper/doggo','None'),('Lila/Maggie','pupper/doggo','None')]
df_twt_arch_c.loc[[748,536,562],'pupper']='None'
df_twt_arch_c.loc[[848,897],['doggo','pupper']]='pupper/doggo','None'

In [ ]:
df_twt_arch_c[(df_twt_arch_c.doggo != 'None') & (df_twt_arch_c.puppo !='None')].index
df_twt_arch_c.loc[154,'doggo']='None'

##### Test

In [ ]:
df_twt_arch_c.iloc[[161,358,416,446,689,748,848,897,154]]

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="A1.6"></a>
#### A.1.6-Dog stage is not properly extracted, 'floof' in tweet text is ignored sometimes and missing values are misrepresented
---
##### Define
Here we have 2 issues that we will address as follows:

- Clear tidiness issue # *B.1.2 One variable in four columns I.e: 'doggo', 'floofer' are different representations of a single variable 'Dog Stage'*
- Search in tweet text and replace dog stage if found in text

In [ ]:
# remove 'None' to properly join dog stage strings
df_twt_arch_c[['doggo','floofer','pupper','puppo']] = df_twt_arch_c[['doggo', 'floofer','pupper','puppo']].replace('None', '')
df_twt_arch_c['dog_stage'] =df_twt_arch_c[df_twt_arch_c.columns[6:]].apply(lambda x: ''.join(x),axis=1)
df_twt_arch_c['dog_stage'].replace('',np.nan)
# drop unwanted columns
df_twt_arch_c.drop(['doggo','floofer','pupper','puppo'],axis=1,inplace=True)

##### Code

In [ ]:
# Search, extract and replace dog stage if found in tweet text
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('doggo') & ~(df_twt_arch_c.dog_stage!='not mentioned'),'dog_stage']='doggo'
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('floof') & ~(df_twt_arch_c.dog_stage!='not mentioned'),'dog_stage']='floofer'
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('pupper') & ~(df_twt_arch_c.dog_stage!='not mentioned'),'dog_stage']='floofer'
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('puppo') & ~(df_twt_arch_c.dog_stage!='not mentioned'),'dog_stage']='floofer'

##### Test

In [ ]:
df_twt_arch_c.dog_stage.value_counts()

In [ ]:
df_twt_arch_c.sample(5)

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('floof') & ~(df_twt_arch_c.dog_stage!='not mentioned')]

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('floof') & ~(df_twt_arch_c.dog_stage!='not mentioned')]

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('pupper') & ~(df_twt_arch_c.dog_stage!='not mentioned')]

In [ ]:
df_twt_arch_c.loc[df_twt_arch_c.text.str.contains('puppo') & ~(df_twt_arch_c.dog_stage!='not mentioned')]

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="A2.2"></a>
#### A.2.2-Image prediction results are somewhat misleading
---
##### Define

We'd like to analyze original ratings of __*dogs*__ only and since we can't fully place reliance on prediction results, due to the existence of several misleading entries; We'll use additional variable to guide our cleaning effort.

- Add 'favorite count', 'retweet count' and 'rating numerator' to the dataframe. Most likely, higher engagement signals that this tweet is for an actual dog.
- Reassess and drop tweets having no dog images
- Drop any unnecessary columns

##### Code

In [ ]:
# add engagement variables (favorite/retweet/rating) from twitter api and archive dataframes
df_img_pred_c=pd.merge(df_img_pred_c,df_twt_api_c[['tweet_id','retweet_count','favorite_count']],on='tweet_id',how='left')
df_img_pred_c=pd.merge(df_img_pred_c,df_twt_arch_c[['tweet_id','rating_numerator']],on='tweet_id',how='left')

In [ ]:
df_img_pred_c.info()

In [ ]:
# drop tweets having no records in the api dataframe, these are most probably deleted tweets
df_img_pred_c.dropna(axis=0,inplace=True)
df_img_pred_c.reset_index(drop=True,inplace=True)
# convert to proper dtype
df_img_pred_c=df_img_pred_c.astype({'favorite_count':int,'retweet_count':int})

In [ ]:
# filter all false predictions
df_false=df_img_pred_c[(df_img_pred_c.p1_dog ==False) & (df_img_pred_c.p2_dog==False) & (df_img_pred_c.p3_dog==False)]
# display tweets with highest engagement values, drop unnecessary columns for more convenience
df_false.sort_values('favorite_count', ascending=False).drop(['img_num','p1_dog','p2_dog','p3_dog'],axis=1).head(10)

In [ ]:
# display tweets with highest rating, drop unnecessary columns for more convenience
df_false.sort_values('rating_numerator', ascending=False).drop(['img_num','p1_dog','p2_dog','p3_dog'],axis=1).head(10)

In [ ]:
# as per the accounts wiki, Nelson sees 30,000 likes on a post as being viral so we will consider this in filtering out false predictions
# keep all false predictions above 30k favorite count and drop those with no dog images
df_false_top_fav=df_false[(df_false.favorite_count > 30000) & ~(df_false.p1.str.contains('web_site|beaver|comic_book|revolver'))]
# keep top false predictions with highest ratings
df_false_top_rat=df_false[(df_false.rating_numerator > 200) & ~(df_false.p1.str.contains('microphone'))]

In [ ]:
# confirm cleaning
df_false_top_fav.sort_values('favorite_count', ascending=False).drop(['img_num','p1_dog','p2_dog','p3_dog'],axis=1)

In [ ]:
# confirm cleaning
df_false_top_rat.sort_values('favorite_count', ascending=False).drop(['img_num','p1_dog','p2_dog','p3_dog'],axis=1)

In [ ]:
# Assessment of true prediction credibility
df_img_pred_c[(df_img_pred_c.p1_dog ==True) & (df_img_pred_c.p2_dog==True) & (df_img_pred_c.p3_dog==True)].sort_values('favorite_count', ascending=False).drop(['img_num','p1_dog','p2_dog','p3_dog'],axis=1).head(10)

In [ ]:
# We will rely on 1st prediction results as 70% are correct.
# drop all false 1st predictions
df_img_pred_c.drop(df_img_pred_c[(df_img_pred_c.p1_dog==False)].index,inplace=True)

# add worng false predictions
df_img_pred_c=df_img_pred_c.append([df_false_top_fav,df_false_top_rat],ignore_index=True)

# rename for proper representation
df_img_pred_c['breed']=df_img_pred_c['p1']

# drop unwanted columns and reset index
df_img_pred_c.drop(['img_num','p1','p1_conf','p1_dog','p2','p2_conf','p2_dog','p3','p3_conf','p3_dog'],axis=1,inplace=True)
df_img_pred_c.reset_index(drop=True,inplace=True)

##### Test

In [ ]:
df_img_pred_c.sort_values('rating_numerator', ascending=False)

In [ ]:
df_img_pred_c.info()

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="A2.3"></a>
#### A.2.3-Some breed names are incorrect and sometimes misleading
---
##### Define
- Unify Breed column's naming convention across the three dataframes this will also clear issue --> *A.3.1 and A.4.1- Breed column's naming convention is not unified* <--

- Add breed info text to make a comprehensive breed dataframe, this will also clear issue --> *B.2.1 All breed information should be included in the Dataframe* <--



##### Code

In [ ]:
# correcting wrong dog breed from both true and false prediction results

# dict of correct breed, built based on visual inspection of tweet content
correct_breed_dict={822872901745569793:'labrador retriever',
                    739238157791694849:'siberian husky',
                    879415818425184262:'basset hound',
                    819004803107983360:'portuguese water dog',
                    819006400881917954:'portuguese water dog',
                    887517139158093824:'dachshund',
                    849051919805034497:'labrador retriever',
                    892420643555336193:'hokkaido',
                    711694788429553666:'golden retriever',
                    852311364735569921:'cavachon',
                    870804317367881728:'portuguese water dog',
                    762035686371364864:'belgian sheepdog',
                    749981277374128128:'australian shepherd',
                    731156023742988288:'Multiple'}

# index list of tweets to be corrected
index_list=df_img_pred_c[df_img_pred_c['tweet_id'].isin(correct_breed_dict.keys())].index

# apply corerction, retrieve correct breed from breed dict that match tweet id found in index list
for i in index_list:
    df_img_pred_c.loc[[i],'breed']=correct_breed_dict[(df_img_pred_c.iloc[i]['tweet_id'])]

In [ ]:
# Unify breed column str format
df_img_pred_c.breed=df_img_pred_c.breed.str.lower().str.replace('_',' ').str.replace('-',' ')
df_breed_c.breed=df_breed_c.breed.str.lower().str.replace('_',' ').str.replace('-',' ')
df_breed_inf_c.breed=df_breed_inf_c.breed.str.replace('-',' ')

# make a unifed breed dataframe
df_img_pred_c=pd.merge(df_img_pred_c,df_breed_c[['breed','breed_info_link','breed_img_link']],on='breed',how='left')

In [ ]:
# identify differences in breed name, nulls are breeds not found in the breed dataframe, I'll use breed name from there as a basis of unification
df_img_pred_c[df_img_pred_c['breed_info_link'].isnull()].groupby(['breed']).groups.keys()

In [ ]:
# use the below line to query correct breed name from breed dataframe
df_breed_c[df_breed_c.breed.str.contains('lhasa')]

In [ ]:
# list of breed names to be unified
wrong_breed_keys_list=df_img_pred_c[df_img_pred_c['breed_info_link'].isnull()].groupby(['breed']).groups.keys()

# list of correct breed name from breed datafram
correct_breed_values_list=['airedale terrier','appenzeller sennenhunde','basset hound','cavalier king charles spaniel','bluetick coonhound',
                                'boston terrier','brussels griffon','brittany','bullmastiff','cairn terrier','cardigan welsh corgi','chow chow','clumber spaniel',
                                'dandie dinmont terrier','doberman pinscher','english springer spaniel','entlebucher mountain dog','american eskimo dog','german shepherd dog',
                                'german shorthaired pointer','Multiple','belgian sheepdog','japanese chin','australian kelpie',
                                'leonberger','lhasapoo','alaskan malamute','belgian malinois','maltese','xoloitzcuintli','poodle', 'Multiple', 'pekingese',
                                'pembroke welsh corgi','redbone coonhound','scottish terrier','staffordshire bull terrier','poodle','poodle','toy fox terrier',
                                'treeing walker coonhound','fox terrier']

len(wrong_breed_keys_list)==len(correct_breed_values_list)

In [ ]:
# building correct breed dict from wrong keys and correct values, wrong keys will be used to index correct values
correct_breed_name_dict=dict(list(zip(wrong_breed_keys_list,correct_breed_values_list)))

# function to apply correct breed name on a dataframe column
def correct_breed_name(df):
    if df['breed'] in correct_breed_name_dict.keys():
        correct_breed = correct_breed_name_dict[df['breed']]
        return correct_breed
    else:
        return df['breed']

# apply function on 'breed' column
df_img_pred_c['breed'] = df_img_pred_c.apply(correct_breed_name, axis=1)

In [ ]:
# update prediction's newly created breeds

# unifying index of both dataframes for update, similar to 'vlookup'
df_img_pred_c.index=df_img_pred_c.breed
df_breed_c.index=df_breed_c.breed

# updated new values
df_img_pred_c.update(df_breed_c,overwrite=False)

# reset index
df_img_pred_c.reset_index(drop=True,inplace=True)
df_breed_c.reset_index(drop=True,inplace=True)

# inspect
df_img_pred_c.head()

In [ ]:
df_img_pred_c.loc[df_img_pred_c['breed_info_link'].isna()]

# these images feature many dogs of mixed breeds, so will keep them referenced as 'Multiple'.

In [ ]:
# add breed info text to the data frame for a quick glance instead of visiting the website
df_img_pred_c=pd.merge(df_img_pred_c,df_breed_inf_c[['breed','breed_info_text']],on='breed',how='left')

# inspect
df_img_pred_c.info()

In [ ]:
# check nulls in new column, other are expected as we have two 'Multiple' breeds
df_img_pred_c[df_img_pred_c.breed_info_text.isna()]

In [ ]:
# whats wrong with 'xoloitzuintli' ???
df_breed_inf_c[df_breed_inf_c.breed.str.contains('xoloitzuintli')]

In [ ]:
# updated with correct name
df_breed_inf_c.loc[382,'breed']='xoloitzcuintli'
df_img_pred_c.index=df_img_pred_c.breed
df_breed_inf_c.index=df_breed_inf_c.breed
df_img_pred_c.update(df_breed_inf_c,overwrite=False)
df_img_pred_c.reset_index(drop=True,inplace=True)
df_breed_inf_c.reset_index(drop=True,inplace=True)

##### Test

In [ ]:
for i in index_list:
    if any (df_img_pred_c.loc[[i],'breed']==correct_breed_dict[(df_img_pred_c.iloc[i]['tweet_id'])]):
        print(i,'----All good, great job!')
    else:
        print('blep!', i)

In [ ]:
df_img_pred_c.info()

In [ ]:
df_img_pred_c.sample(5)

In [ ]:
df_img_pred_c[df_img_pred_c['breed_info_link'].isnull()]

**<a href="#qlt" style='color:#8D38C9'>Quality Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="tidinessissues"></a>
### Tidiness Issues

<a class="anchor" id="B1.1"></a>
#### B.1.1-Tweet text column include both tweet text and link
---
##### Define

- Split tweet text and link into separate columns
- Bouns: change header'text' to 'tweet_text'


##### Code

In [ ]:
# extract, split and merge
df_splitted_columns=df_twt_arch_c.text.str.split(r'(https?:(/{1,3})\S+)',expand=True)
df_twt_arch_c[['text','tweet_link']]=df_splitted_columns[[0,1]]
df_twt_arch_c.rename(columns={'text':'tweet_text'},inplace=True)

##### Test

In [ ]:
df_twt_arch_c.head()

In [ ]:
df_twt_arch_c.info()

In [ ]:
# whether to grab link for these tweets or not will be decided after cleaning next issue # B.1.3
df_twt_arch_c[df_twt_arch_c.tweet_link.isnull()]

**<a href="#tid" style='color:#8D38C9'>Tidiness Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="B1.3"></a>
#### B.1.3-Total entries does not match that in Image prediction  Dataframe
---
##### Define
Since our defined schema for this analysis is original ratings for dogs only, we'll:

- Drop tweets from cleaned tweet archive that are not found in image prediction dataframe



##### Code

In [ ]:
df_twt_arch_c=df_twt_arch_c[df_twt_arch_c.tweet_id.isin(df_img_pred_c.tweet_id)]
df_twt_arch_c.reset_index(drop=True,inplace=True)

##### Test

In [ ]:
df_twt_arch_c.info()

In [ ]:
df_img_pred_c.info()

In [ ]:
df_img_pred_c[~df_img_pred_c.tweet_id.isin(df_twt_arch_c.tweet_id)].sort_values('tweet_id',ascending=False)

In [ ]:
# Tweets missing links are already droped
df_twt_arch_c[df_twt_arch_c.tweet_link.isnull()]

**<a href="#tid" style='color:#8D38C9'>Tidiness Issues</a>**

**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="masterdataframes"></a>
### Master DataFrame(s)

We'll create two main dataframes one for tweets another for breeds and will keep tweet id as well as favorite count in both. Considering that we'd like to analyze various drivers of higher engagement, it will be convenient if we have these figures in front of us in both dataframes.


As we start analyzing data we will load main dataframes to sub ones (Doggo Tweets/Breeds), sorting and dropping columns that won't be used much in our analysis.


* <a href="#maindf" style='color:#8D38C9'>Main Dataframe(s)</a>

* <a href="#doggotweet" style='color:#8D38C9'>Sub-Doggo Tweets</a>

* <a href="#doggobreed" style='color:#8D38C9'>Sub-Doggo Breeds</a>

<a class="anchor" id="maindf"></a>
### Main Dataframe(s)

##### Code

In [ ]:
# merge
df_twt_arch_c=pd.merge(df_twt_arch_c,df_twt_api_c[['tweet_id','retweet_count','favorite_count']],on='tweet_id',how='left')
# split observational units (tweet/breed) in separate dataframes
df_img_pred_c=pd.merge(df_img_pred_c,df_twt_arch_c[['tweet_id','dog_name','dog_stage']],on='tweet_id',how='left')

In [ ]:
# drop unnecessary columns
df_twt_arch_c.drop(columns=['dog_name','dog_stage'], inplace=True)
df_img_pred_c.drop(columns=['retweet_count','rating_numerator'], inplace=True)

# rename for better representation
df_twt_arch_c.rename(columns={'rating_numerator':'dog_rating'},inplace=True)
df_img_pred_c.rename(columns={'jpg_url':'dog_picture'},inplace=True)

In [ ]:
# save master dataframe(s)
df_twt_arch_c.to_csv('twitter_archive_master.csv',index=False)
df_img_pred_c.to_csv('breed_master.csv',index=False)

##### Test

In [ ]:
df_twt_arch_c.info()

In [ ]:
df_img_pred_c.info()

In [ ]:
df_twt_arch_c.head()

In [ ]:
df_img_pred_c.head()

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**    
**<a href="#analysis" style='color:#8D38C9'>Analysis and visualization</a>**  
**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="doggotweet"></a>
### Doggo Tweets

In [ ]:
# Load data 
df_twt_arch_c=pd.read_csv('twitter_archive_master.csv')
# Make copies
doggos_tweets_df=df_twt_arch_c.copy()

In [ ]:
doggos_tweets_df.info()

In [ ]:
doggos_tweets_df['tweet_time_stamp']=pd.to_datetime(doggos_tweets_df['tweet_time_stamp']).dt.tz_localize(None)

In [ ]:
# list of columns to rearrange, tweets dataframe
columns_tweets=doggos_tweets_df.columns.tolist()
columns_tweets

In [ ]:
# rearrange columns
columns_sort_tweets=columns_tweets[:3]+columns_tweets[5:6]+columns_tweets[6:8]+columns_tweets[3:4]
doggos_tweets_df=doggos_tweets_df[columns_sort_tweets]

In [ ]:
doggos_tweets_df.info()

In [ ]:
doggos_tweets_df.sort_values('favorite_count',ascending=False).head()

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**    
**<a href="#analysis" style='color:#8D38C9'>Analysis and visualization</a>**  
**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="doggobreed"></a>
### Doggo Breeds

In [ ]:
# Load data
df_img_pred_c=pd.read_csv('breed_master.csv')
# Make copy
doggos_breeds_df=df_img_pred_c.copy()

In [ ]:
doggos_breeds_df.info()

In [ ]:
# list of columns to rearrange, breeds dataframe
columns_breeds=doggos_breeds_df.columns.tolist()
columns_breeds

In [ ]:
# rearrange columns, remove those that won't be used much in our analysis
columns_sort_breeds=columns_breeds[:1]+columns_breeds[2:3]+columns_breeds[7:9]+columns_breeds[1:2]+columns_breeds[3:5]
doggos_breeds_df=doggos_breeds_df[columns_sort_breeds]

In [ ]:
doggos_breeds_df.info()

In [ ]:
doggos_breeds_df.sort_values('favorite_count',ascending=False).head()

**<a href="#assessmentoutcome" style='color:#8D38C9'>Assessment outcome</a>**      
**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="analysis"></a>
### Analysis and visualization

* <a href="#time" style='color:violet'>Account activity over time</a>

* <a href="#tweet" style='color:violet'>Key drivers of followers engagement</a>


**<a href="#jmp2lnks" style='color:#8D38C9'>Notebook Head</a>**

<a class="anchor" id="time"></a>

### Account activity over time

- [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs) account started in 2015 where peak creation of monthly tweets took place as 30% of tweets analyzed where created in the last two months of 2015. The scope of this analysis is the period starting Nov'15 till Aug'17 during which period new tweets creation was in continuous decline; However, followers' engagement in the form of favorits and retweets took a steady increase over the same period.

- Decline in new tweet creation could be directly contributed to the filtration process, as noted in the account's wiki page, the account receive 800 to 1,000 submissions daily and work to narrow them down to about one high-quality piece of dog content per day. It's actually a sign of effective filtration process as few number of tweets generate higher engagement.

- We can see the effect of this 'High-quality' content filteration strategy in the time series plot below. On May'16, June'16, August'16 single tweets caused spikes in user engagement trend and as of August a learning curve seems to have been established as user engagement took a steady increase till the end of the period. We'll explore later key drivers of followers engagement but we can conclude that content and context of the tweet plays a major role. Check these tweets, [1st](https://t.co/7wE9LTEXC4), [2nd](https://t.co/nTz3FtorBc), [3rd](https://t.co/DkBYaCAg2d), [5th](https://t.co/YcXgHfp1EC), [7th](https://t.co/AdPKrI8BZ1) ranks represent favorite count from highest to lowest, let's see if we both reached the same conclusion as you read on through the rest of this analysis.

- It's clear that favorite as a type of engagment is taking precedence over retweet for this particular account, however, both have a positive correlation. It actually makes sense as favorite in general is a sign of appreciation and appeal which conforms to the context of these tweets.

- Another interesting observation is that two independent yet identical tweets received different follower reactions. The [first](https://twitter.com/dog_rates/status/670668383499735048) posted on Nov'15 had a total favorite count of almost 10k, while the [second](https://twitter.com/dog_rates/status/892420643555336193) posted on Aug'17 had a total of 35K!. Setting aside that the tweet deserve higher engagement, I like donuts so this is a biased statement and also the doggo is cute, this highlights an effective practice of account management by seeking increased engagement through increasing tweet reach.

In [ ]:
# separating year and month for further analysis
doggos_tweets_df['year'] = doggos_tweets_df['tweet_time_stamp'].dt.year
doggos_tweets_df['month'] = doggos_tweets_df['tweet_time_stamp'].dt.month

# # execluded aug'17 since we have only two records for this month
time_interval_df=doggos_tweets_df[~((doggos_tweets_df.year == 2017) & (doggos_tweets_df.month == 8))]

doggos_tweets_df.info()

In [ ]:
# Viral tweets timeframe
time_interval_df.sort_values('favorite_count', ascending=False).head(10)

In [ ]:
# Number of new tweets overtime
time_interval_df.groupby(['year','month'])['tweet_id'].count()

In [ ]:
# Engagement trend overtime
time_interval_df.groupby(['year','month'])['favorite_count'].max()

In [ ]:
# setup plot
fig, ax = plt.subplots(1,1,figsize=(15,5), dpi=80)
fig.suptitle('Account activity overtime', c='rosybrown', fontsize=15, ha= 'center')
fig.subplots_adjust(top=0.8)
fig.tight_layout()


# Plot Line1 (Left Y Axis), resample timestamp by monthly bins
time_interval_df.groupby(['tweet_time_stamp'])['tweet_id'].count().resample('1m').count().plot(ax=ax,c='red', label='New Tweet Count');

# instantiate a second axes that shares the same x-axis
ax2 = ax.twinx() 

# Plot Line2 (Right Y Axis), resample timestamp by monthly bins
time_interval_df.groupby(['tweet_time_stamp'])['favorite_count'].mean().resample('1m').mean().plot(ax=ax2,c='purple', label='Average favorite Count');

# instantiate a third axes that shares the same x-axis
ax3 = ax.twinx()  

# Plot Line3 (Right Y Axis), resample timestamp by monthly bins
time_interval_df.groupby(['tweet_time_stamp'])['retweet_count'].mean().resample('1m').mean().plot(ax=ax3,c='orange', label='Average retweet Count');


# Decoration
ax.set_xlabel("Time interval", c='chocolate', fontsize=15, x=.5)
ax.set_ylabel("Tweet Count", c='red', fontsize=15); ax2.set_ylabel("Average favorite and retweet Count", c='purple', fontsize=15, x=.5)
ax.grid(alpha=.3)
ax3.axis('off')
ax.spines["top"].set_alpha(0.0); ax2.spines["top"].set_alpha(0.0);ax3.spines["top"].set_alpha(0.0)
ax.spines["top"].set_alpha(0.0); ax2.spines["top"].set_alpha(0.0);ax3.spines["top"].set_alpha(0.0)
ax.legend(loc=4, bbox_to_anchor=(1, .10)); ax2.legend(loc=4, bbox_to_anchor=(1, .95)); ax3.legend(loc=4, bbox_to_anchor=(1, .30))
ax3.set_yticks(np.linspace(ax2.get_yticks()[0], ax2.get_yticks()[-1], len(ax3.get_yticks())));

<a class="anchor" id="tweet"></a>

### Key drivers of followers engagement  

*  while dog name and stage may be included in each tweet this is not always the case, the following represent minimum content of each tweet:
   - Media either picture or video
   - Descriptive text
   - Rating

- We've incorporated image prediction results into our analysis;  these predictions are outputs of a neural network that can classify breeds of dogs based on their images. By comparing the relative featuring frequency of a specific breed to the corresponding engagement matrices which is in our case __*favorite count*__, we concluded that breed alone does not have a material impact on user engagement. For instance, [Golden retriever](https://dogtime.com/dog-breeds/golden-retriever) breed was featured 138 times but highest favorite count received for a single tweet was 76k while [Portuguese water dog](https://dogtime.com/dog-breeds/portuguese-water-dog) was featured only 3 times with highest favorite count of 85k; [French bulldog](https://dogtime.com/dog-breeds/french-bulldog), [Siberian husky](https://dogtime.com/dog-breeds/siberian-husky) and [Labrador retriever](https://dogtime.com/dog-breeds/labrador-retriever) were featured 25/21/94 times with highest favorite count of 112k/111k/150k respectively. It worth mentioning that image prediction results are not always correct but we did our best filtering out incorrect results specially for outliers and tweets having high user engagement.


- Same as for dog rating, no material effect on user interactions because at the end of the day [They're good dogs Brent](https://www.vox.com/2018/7/23/17603566/dog-rates-good-dogs-brent-brant-got-a-puppy-meme) !. Average rating for tweets included in this data set is 13 where 75% of tweets have a rating of 12, only two outliers though and the top one have an interesting story behind it. [Atticus](https://thedailyatticus.wordpress.com/atticus/) was given highest rating of all time being 1776 , surprisingly the [tweet](https://t.co/GRXwMxLBkh) had relatively low user engagement of almost 5k favorite and 2k retweets. The tweet was created on 4th of July'16 as a celebration of US declaration of Independence day that took place on 1776.


* What's driving higher user engagement? by exploring top favorite tweets and zooming in the outliers we can conclude that: 
  - [Videos](https://databox.com/videos-vs-images-in-facebook-ads) are the star of the show, 80% of top favorite tweets featured videos.
  - Puppers and puppos (small dogs) shine, either alone or with another doggo. In fact, tweets featuring puppos received higher average rating than their counter parts.
  - Doggos' biography counts!. Actually the reason behind why Portuguese water dog breed gained traction while being only featured three times is the fact that 2 of the three tweets featured ex US First doggo ['Sunny'](https://en.wikipedia.org/wiki/Sunny_(dog)) and his sister 'Bo'
  - Pose matters. Check out these pictures from top favorite tweets [Pupper/Doggo](https://twitter.com/dog_rates/status/733109485275860992), [Puppo](https://twitter.com/dog_rates/status/889665388333682689), [Pupper](https://twitter.com/dog_rates/status/866450705531457537), [Zoey](https://twitter.com/dog_rates/status/870374049280663552), [Aja](https://t.co/lsPyyAiF1r). [Honey](https://twitter.com/dog_rates/status/1370171823829049344) is epic, it's a recent tweet though and not from the data set being analyzed.
  - Timing makes a difference. 2nd top favorite tweet is for a [doggo](https://twitter.com/dog_rates/status/822872901745569793) participating in the 2017 [Women's March](https://en.wikipedia.org/wiki/2017_Women%27s_March)

- So as we mentioned earlier, content and context influence user engagement. Tweets composed of a good mixture of both are most likely to receive higher user engagement, this also explains the reason why some tweets featuring same dog breed for example received widely varying favorite counts than others. If you are a dog owner seeking to celebritize your doggo then take this into consideration in your next photo session

- [Golden retriever](https://dogtime.com/dog-breeds/golden-retriever) is the top featured breed in this dataset with a total featuring frequency of 138 times, it's one of the most popular dog breeds in the United States. The breed’s friendly, tolerant attitude makes them great family pets, and their intelligence makes them highly capable working dogs.

In [ ]:
doggos_tweets_df.dog_rating.describe().round()

In [ ]:
# top rated doggos
doggos_tweets_df.sort_values('dog_rating',ascending=False).head(10)

In [ ]:
# Viral tweets
doggos_tweets_df.sort_values('favorite_count',ascending=False).head(10)

In [ ]:
# building a dataframe for breed being the observational unit and favorite count/frequency as variables. focusing on favorite count being key engagement metrics

df_1=doggos_breeds_df.groupby(['breed'],as_index=False)['favorite_count'].max().round().reset_index().rename(columns={'favorite_count':'max_fav_count'})
df_2=doggos_breeds_df.groupby(['breed'],as_index=False)['favorite_count'].min().round().reset_index().rename(columns={'favorite_count':'min_fav_count'})
df_3=doggos_breeds_df.groupby(['breed'],as_index=False)['favorite_count'].mean().round().reset_index().rename(columns={'favorite_count':'avg_fav_count'})
df_4=doggos_breeds_df.groupby(['breed'],as_index=False)['favorite_count'].sum().round().reset_index().rename(columns={'favorite_count':'total_fav_count'})
df_5=doggos_breeds_df.breed.value_counts().to_frame().reset_index().rename(columns={'index':'breed','breed':'featuring_frequency'})

# list of df to merge
data_frames_list=[df_1,df_2,df_3,df_4,df_5]

# merge
breed_stats_df = reduce(lambda  left,right: pd.merge(left,right,on=['breed'],how='outer'), data_frames_list)

# drop extra columns
breed_stats_df.drop(columns=['index_x','index_y'],axis=1,inplace=True)

In [ ]:
breed_stats_df.info()

In [ ]:
# viral tweets stats in relation to breed category
breed_stats_df.sort_values('max_fav_count', ascending=False).head(10)

In [ ]:
# query data from tweets master data frame
query_df=doggos_breeds_df[doggos_breeds_df.breed.str.contains('labrador retriever')]
doggos_tweets_df[doggos_tweets_df.tweet_id.isin(query_df.tweet_id)].sort_values('favorite_count', ascending=False).head()

In [ ]:
# Prepare Data

# top 10 viral tweets categorized by breeds
top_breeds_df = breed_stats_df[breed_stats_df.index.isin(breed_stats_df.sort_values('max_fav_count', ascending=False).head(10).index)]


# setup plot
fig, ax = plt.subplots(1,1,figsize=(15,5), dpi=80)
fig.autofmt_xdate(rotation=45)

# Number of bars, must match number of rows
x = np.arange(10)
# width 
w = 0.3

# Plot Bars
fav_count =ax.bar(x, top_breeds_df['max_fav_count'], width=w, color='sandybrown', align='center')

#instantiate a 2nd axes that shares the same x-axis
ax2 = ax.twinx()

feat_freq =ax2.bar(x + w, top_breeds_df['featuring_frequency'], width=w, color='rosybrown',align='center')

# Decoration
ax.set_ylabel("Maximum favorite count", c='sandybrown', fontsize=15); ax2.set_ylabel("Featuring frequency", c='rosybrown', fontsize=15)
ax.spines["top"].set_alpha(0.0); ax2.spines["top"].set_alpha(0.0)
plt.xticks(x + w /2, top_breeds_df.breed.str.capitalize())
plt.title("Top 10 featured doggos breeds vs maximum favorite count", c='chocolate', fontsize=15, ha= 'center', y=1.02)
plt.legend([fav_count, feat_freq],['Maximum favorite count', 'Featuring frequency']);

In [ ]:
# execulde missing names and stages
doggo_name_df=doggos_breeds_df[~(doggos_breeds_df.dog_name.isnull())]
doggo_stage_df=doggos_breeds_df[~(doggos_breeds_df.dog_stage.isnull())]

In [ ]:
# Prepare Data

# building a dataframe for stage being the observational unit and favorite count/frequency as variables.

df_1 = doggo_stage_df.dog_stage.value_counts().to_frame().reset_index().rename(columns={'index':'dog_stage','dog_stage':'featuring_frequency'})
df_2 = doggo_stage_df.groupby(['dog_stage'],as_index=False)['favorite_count'].max().round().reset_index().rename(columns={'favorite_count':'max_fav_count'})

data_frames_list=[df_1,df_2]
stage_stats_df = reduce(lambda  left,right: pd.merge(left,right,on=['dog_stage'],how='outer'), data_frames_list)
stage_stats_df.drop(columns=['index'],axis=1,inplace=True)

# Plot Bars
fig, ax = plt.subplots(1,1,figsize=(15,5), dpi=80)

# Number of bars, must match number of rows
x = np.arange(5)
# width
w = 0.3

fav_count =ax.bar(x, stage_stats_df['max_fav_count'], width=w, color='burlywood', align='center')

#instantiate a 2nd axes that shares the same x-axis
ax2 = ax.twinx()
feat_freq =ax2.bar(x + w, stage_stats_df['featuring_frequency'], width=w,color='brown',align='center')


# Decoration

ax.set_ylabel("Maximum favorite count", color='burlywood', fontsize=15); ax2.set_ylabel("Featuring frequency", color='brown', fontsize=15)
ax.spines["top"].set_alpha(0.0); ax2.spines["top"].set_alpha(0.0)
plt.xticks(x + w /2, stage_stats_df.dog_stage.str.capitalize())
plt.title("Top featured doggo stage vs maximum favorite count", c='chocolate', fontsize=15, ha= 'center', y=1.02)
plt.legend([fav_count, feat_freq],['Maximum favorite count', 'Featuring frequency']);

In [ ]:
# Prepare Data

# breeds featured more than 10 times
df = doggos_breeds_df.breed.value_counts().to_frame().reset_index().rename(columns={'index':'breed','breed':'featuring_frequency'})
featuring_frequency_df = df[df.featuring_frequency > 10]

# setup plot labels, box sizes and colors
labels = featuring_frequency_df.apply(lambda x: str(x[0]) + "\n ("+ str(x[1]) + ")", axis=1)
sizes = featuring_frequency_df['featuring_frequency'].values.tolist()
colors = [plt.cm.Spectral(i/float(len(labels))) for i in range(len(labels))]

# Draw Plot
plt.figure(figsize=(30,14), dpi= 80)
squarify.plot(sizes=sizes, label=labels, color=colors, alpha=1, bar_kwargs=dict(linewidth=2, edgecolor="#222222"), text_kwargs=dict(fontsize=11));

# Decorate
plt.title('Doggo breeds featuring frequency', fontsize=20, y=1.01, c='brown')
plt.axis('off');

In [ ]:
# Prepare Data

# featuring frequency
df = doggo_name_df.dog_name.value_counts().to_frame().reset_index().rename(columns={'index':'dog_name','dog_name':'featuring_frequency'})

# excluding single occurrence
frequent_names_df=df[df.featuring_frequency>3].sort_values('dog_name')

# number of rows for coloring each bars
n = frequent_names_df['dog_name'].unique().__len__()+1

# all colors are stored in the below list, use it to explore different ones
all_colors = list(plt.cm.colors.cnames.keys())

# setting up colors of each bar
random.seed(len(frequent_names_df))
c = random.choices(all_colors, k=n)

# Plot Bars
plt.figure(figsize=(16,10), dpi= 80)
plt.bar(frequent_names_df['dog_name'], frequent_names_df['featuring_frequency'], color=c, width=.6)

# displaying y axis numbers on top of the bars
for i, val in enumerate(frequent_names_df['featuring_frequency'].values):
    plt.text(i, val, float(val), horizontalalignment='center', verticalalignment='bottom', fontdict={'fontweight':500, 'size':12})

# Decoration
plt.gca().set_xticklabels(frequent_names_df['dog_name'], rotation=60, horizontalalignment= 'right')
plt.title("Top featured doggo names", fontsize=22, c='brown')
plt.ylabel('featuring frequency')
plt.ylim(0, 11)
plt.gca().spines["top"].set_alpha(0.0); plt.gca().spines["right"].set_alpha(0.0)

###### Male doggos dominate, not fair !

**<a href="#analysis" style='color:#8D38C9'>Analysis and visualization</a>**